In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
df_mkt = (
    session.read.option("FIELD_OPTIONALLY_ENCLOSED_BY", '"')
    .csv("@stg_raw/marketing_campaign_dataset.csv")
    .to_pandas()
)

In [ ]:
df_mkt.head(20)

In [ ]:
print("Row marketing:", len(df_mkt))
df_mkt.info()

In [ ]:
# Convert row-0 as header
df_mkt.columns = df_mkt.iloc[0]
df_mkt = df_mkt[1:].reset_index(drop=True)

#convert header to lowercase
df_mkt.columns = df_mkt.columns.str.strip().str.lower()

In [ ]:
df_mkt.head()

In [ ]:
#Casting Datatype
# Convert numeric fields
numeric_cols = ["conversion_rate", "acquisition_cost", "roi", "clicks", "impressions", "engagement_score"]
for col in numeric_cols:
    if col in df_mkt.columns:
        df_mkt[col] = pd.to_numeric(df_mkt[col].str.replace(r'[\$,]', '', regex=True), errors='coerce')

# Duration to numeric
if "duration" in df_mkt.columns:
    df_mkt["duration"] = df_mkt["duration"].str.extract(r'(\d+)').astype(float).astype("Int64")

# Date to datetime
if "date" in df_mkt.columns:
    df_mkt["date"] = pd.to_datetime(df_mkt["date"], errors="coerce")
    df_mkt["campaign_year"] = df_mkt["date"].dt.year
    df_mkt["campaign_month"] = df_mkt["date"].dt.month
    df_mkt["campaign_quarter"] = df_mkt["date"].dt.quarter
    df_mkt["campaign_weekday"] = df_mkt["date"].dt.day_name()
    
    df_mkt["date"] = df_mkt["date"].dt.strftime("%Y-%m-%d %H:%M:%S")


In [ ]:
print(df_mkt.dtypes)

In [ ]:
df_mkt.head()

In [ ]:
#Outlier Removal
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower = Q1 - 1.5*IQR
        upper = Q3 + 1.5*IQR
        
        df = df[(df[col] >= lower) & (df[col] <= upper)]
        
    return df

mkt_outlier_cols = [
    "duration", 
    "conversion_rate",
    "acquisition_cost",
    "roi"
]

df_mkt = remove_outliers_iqr(df_mkt, mkt_outlier_cols)
print("Marketing rows after:", len(df_mkt))


In [ ]:
df_mkt.head()

Feature Engineering

In [ ]:


# Fitur rasio dan efisiensi
df_mkt["roi_per_cost"] = df_mkt["roi"] / (df_mkt["acquisition_cost"] + 1e-6)
df_mkt["engagement_ratio"] = df_mkt["clicks"] / (df_mkt["impressions"] + 1e-6)
df_mkt["cost_per_click"] = df_mkt["acquisition_cost"] / (df_mkt["clicks"] + 1e-6)
df_mkt["cost_per_engagement"] = df_mkt["acquisition_cost"] / (df_mkt["engagement_score"] + 1e-6)

# Fitur kombinasi bisnis
df_mkt["audience_type"] = df_mkt["target_audience"] + "_" + df_mkt["customer_segment"]

# Tentukan threshold otomatis
roi_thr = df_mkt["roi"].mean()
eng_thr = df_mkt["engagement_score"].mean()
cost_thr = df_mkt["acquisition_cost"].mean()

# Fitur kategorikal tambahan

df_mkt["roi_level"] = pd.qcut(df_mkt["roi"], q=3, labels=["Low ROI", "Medium ROI", "High ROI"])
df_mkt["engagement_level"] = pd.qcut(df_mkt["engagement_score"], q=3, labels=["Low Eng", "Medium Eng", "High Eng"])
df_mkt["cost_efficiency"] = pd.qcut(df_mkt["acquisition_cost"], q=3, labels=["High Cost", "Medium Cost", "Low Cost"])

df_mkt["is_high_roi"] = (df_mkt["roi"] > roi_thr).astype(int)
df_mkt["is_high_engagement"] = (df_mkt["engagement_score"] > eng_thr).astype(int)
df_mkt["is_cost_efficient"] = (df_mkt["acquisition_cost"] < cost_thr).astype(int)

# Indeks performa gabungan
df_mkt["performance_index"] = (
    df_mkt["is_high_roi"]
    + df_mkt["is_high_engagement"]
    + df_mkt["is_cost_efficient"]
)


# Indeks performa gabungan
df_mkt["performance_index"] = (
    df_mkt["is_high_roi"]
    + df_mkt["is_high_engagement"]
    + df_mkt["is_cost_efficient"]
)


In [ ]:
df_mkt.head()

In [ ]:
# Save Dataframe to Table
session.write_pandas(df_mkt, "MARKETING_CAMPAIGN", auto_create_table=True, overwrite=True)


In [ ]:
print(df_mkt.dtypes)